# Model Evaluation - Proje: Kapsamlı Model Karşılaştırması

Bu proje, farklı ML algoritmalarını kapsamlı bir şekilde değerlendirir ve en iyi modeli seçer.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, confusion_matrix, classification_report)
from sklearn.datasets import make_classification

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


## 1. Veri Hazırlama ve Model Eğitimi


In [ ]:
# Veri seti
X, y = make_classification(n_samples=2000, n_features=20, n_informative=15, 
                           n_redundant=5, n_clusters_per_class=1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modeller
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Tüm modelleri eğit ve değerlendir
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba),
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'Model': model,
        'Predictions': y_pred,
        'Probabilities': y_pred_proba
    }

results_df = pd.DataFrame(results).T.drop(['Model', 'Predictions', 'Probabilities'], axis=1)
print("Model Comparison Results:")
print(results_df.round(4))


## 2. Görselleştirmeler


In [ ]:
# Metrik karşılaştırması
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    results_df[metric].plot(kind='bar', ax=ax, color='steelblue')
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel(metric)
    ax.set_xticklabels(results_df.index, rotation=45, ha='right')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ROC Curves
plt.figure(figsize=(10, 8))
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['Probabilities'])
    plt.plot(fpr, tpr, label=f"{name} (AUC = {result['ROC-AUC']:.4f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 3. En İyi Model Seçimi


In [ ]:
# En iyi modeli seç (ROC-AUC'ye göre)
best_model_name = results_df['ROC-AUC'].idxmax()
best_model = results[best_model_name]['Model']

print("=" * 50)
print("EN İYİ MODEL")
print("=" * 50)
print(f"Model: {best_model_name}")
print(f"ROC-AUC: {results_df.loc[best_model_name, 'ROC-AUC']:.4f}")
print(f"Accuracy: {results_df.loc[best_model_name, 'Accuracy']:.4f}")
print(f"F1-Score: {results_df.loc[best_model_name, 'F1-Score']:.4f}")

# En iyi model için detaylı değerlendirme
y_pred_best = results[best_model_name]['Predictions']
print(f"\nConfusion Matrix:\n{confusion_matrix(y_test, y_pred_best)}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred_best)}")
